In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error,accuracy_score

In [2]:
#now we do it for football.csv file
df = pd.read_csv('./titatnic.csv')
df=df.drop('Passengerid',axis=1)
df=df.dropna(axis=0)
df.corr()
y = df.pop('2urvived').to_numpy()
df=df.drop(['zero','zero.1','zero.2','zero.3','zero.4','zero.5','zero.6','zero.7','zero.8','zero.9','zero.10','zero.11','zero.12','zero.13','zero.14','zero.15','zero.16','zero.17','zero.18'],axis=1)
X = df.to_numpy() 
scaler = MinMaxScaler()
X = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [44]:
def sigmoid(z):
        return 1 / (1 + np.exp(-z))

In [85]:
layer_sizes=[7,5,3]
weights=[]
biases=[]
for i in range(1, len(layer_sizes)):
    weights.append(np.random.rand(layer_sizes[i], layer_sizes[i-1]))  # Weights for each layer
    biases.append(np.random.rand(layer_sizes[i])) 

In [72]:
for i in range(len(biases)):
    print(X_train.shape,weights[i].shape,biases[i].shape)
    z[i] = np.dot(X, weights[i]) + biases[i]# find z for the ith layer
    a[i] = sigmoid(z[i]) # find a of the ith layer
    x = a[i] 

(1045, 7) (7,) ()
(1045, 7) (7,) ()
(1045, 7) (7,) ()


In [97]:
class NeuralNet:
    def __init__(self,layer_sizes, learning_rate):
        self.weights = []
        self.biases = []
        for i in range(1, len(layer_sizes)):
            self.weights.append(np.random.rand(layer_sizes[i], layer_sizes[i-1]))  # Weights for each layer
            self.biases.append(np.random.rand(layer_sizes[i]))  # Bias for each layer
        self.learning_rate = learning_rate
        self.z = [None] * len(self.biases)
        self.a = [None] * len(self.biases)
    def forward(self, x):
        for i in range(len(self.biases)):
            print(x.shape,self.weights[i].shape,self.biases[i].shape)
            self.z[i] = np.dot(x, self.weights[i]) + self.biases[i]# find z for the ith layer
            self.a[i] = self.sigmoid(self.z[i]) # find a of the ith layer
            x = self.a[i]  #set input as a of this layer so the next layer uses previous layers activations as input
        return self.a[-1]

    def predict(self, x):
        return self.forward(x)

    def optimize(self, x, y):
        n=len(y)
        for i in reversed(range(len(self.weights))):
            if i == len(self.weights) - 1:
                # For the output layer, use the direct error
                error = self.a[i] - y
            else:
                # For hidden layers, propagate the error from the next layer
                error = np.dot(self.weights[i + 1], error)

            if i != 0:
                dw = np.dot(self.a[i-1].T, error) / n
                db = np.sum(error, axis=0) / n
            else:
                # For the first layer (i = 0), use the input features 'x' instead of previous activations
                dw = np.dot(x.T, error) / n
                db = np.sum(error, axis=0) / n
            self.weights[i] -= self.learning_rate * dw
            self.biases[i] -= self.learning_rate * db


    @staticmethod
    def relu(z):
        return np.maximum(0,z)
    
    @staticmethod
    def sigmoid(z):
        return 1 / (1 + np.exp(-z))

In [102]:
X_train.shape

(1045, 7)

In [100]:
nn=NeuralNet(layer_sizes=[X_train.shape[1],5,3],learning_rate=0.1)
steps=10
for i in range(steps):
    nn.forward(X_train)
    nn.optimize(X_train,y_train)
    pred=nn.predict(X_test)
    y_pred = np.where(pred >= 0.5, 1, 0)
    print(accuracy_score(y_pred,y_test))

(1045, 7) (5, 7) (5,)


ValueError: shapes (1045,7) and (5,7) not aligned: 7 (dim 1) != 5 (dim 0)